<a href="https://colab.research.google.com/github/KNUckle-llm/experiments/blob/main/Q%26A_ChatBot2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 인공지능 PDF Q&A 챗봇 프로젝트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install python-dotenv
# 일단 종호의 openai API 키를 .env 파일에 넣어 놓음 => 나중에 바꾸기

In [2]:
#!pip uninstall -y gradio
#!pip install gradio==3.50.2 --force-reinstall --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 216.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 288.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 275.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 275.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 307.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 344.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 372.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 kB 364.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 319.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 160.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install langchain_openai==0.3.7

In [ ]:
!pip install langchain-huggingface==0.1.2

In [ ]:
!pip install langchain_community==0.3.18

In [ ]:
!pip install faiss-cpu==1.10.0

In [7]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 16.0 MB/s eta 0:00:00


In [14]:
import gradio as gr
import os
import tempfile
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough

# 환경 변수 불러오기(openai API 키)
load_dotenv('/content/drive/MyDrive/Colab Notebooks/.env')

# LLM 설정
llm = ChatOpenAI(model="gpt-4o-mini")

# 텍스트 분리
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

# 임베딩 모델
hf_embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# 프롬프트 템플릿
message = """
당신은 사용자의 질문에 답변을 하는 친절한 AI 어시스턴트입니다.
당신의 임무는 주어진 문맥을 토대로 사용자 질문에 답하는 것입니다.
만약, 문맥에서 답변을 위한 정보를 찾을 수 없다면 '주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다' 라고 답하세요.
정보를 찾을 수 있다면 한글로 답변해 주세요.

## 주어진 문맥:
{context}

## 사용자 질문:
{input}
"""
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("human", message)
    ]
)

# 출력 파서
parser = StrOutputParser()

# 전역 변수
db = None
retriever = None
rag_chain = None
faiss_path = "/content/drive/MyDrive/FaissDB/knu_faiss_db"

# 사용자 질문에 대한 응답을 만들기
def setup_chain():
    global retriever, rag_chain

    retriever = db.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 3, "fetch_k": 10, "lamda_mult": 0.5}
    )

    rag_chain = {
        "context": retriever,
        "input": RunnablePassthrough()
    } | prompt_template | llm | parser

# 드라이브에 존재하는 DB 로드
def load_faiss_db():
    global db
    db = FAISS.load_local(
        folder_path=faiss_path,
        embeddings=hf_embeddings,
        allow_dangerous_deserialization=True
    )
    setup_chain()

# PDF 업로드 및 DB 저장
def add_pdf_to_db(file):
    global db

    loader = PyPDFLoader(file.name)
    docs = loader.load_and_split(text_splitter=text_splitter)

    # 각 청크에 파일명 metadata 추가
    for doc in docs:
        doc.metadata["file_name"] = os.path.basename(file.name)
        # 여기에 url도 넣을 수 있을듯?

    if db is None:
        db = FAISS.from_documents(docs, hf_embeddings)
    else:
      db.add_documents(docs)

    db.save_local(faiss_path)
    setup_chain()

    return f"{os.path.basename(file.name)} 문서를 처리하여 FAISS DB에 저장했습니다."

# 질문 처리
def answer_question(question):
    if rag_chain is None:
        return "먼저 PDF 파일을 업로드하세요!"
    return rag_chain.invoke(question)

def show_stored_documents():
    if db is None:
        return "DB 로드 문제"

    docs = list(db.docstore._dict.values())  # 저장된 모든 청크들을 가져와 리스트로 변환
    file_names = {doc.metadata.get("file_name", "Unknown") for doc in docs}
    return "📚 저장된 문서 목록:\n" + "\n".join(f"• {f}" for f in sorted(file_names))

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 📄 인공지능 PDF Q&A 챗봇
    **여러 PDF 파일을 업로드하고 질문을 입력하면 AI가 답변을 제공합니다!**
    """)

    with gr.Row():
        with gr.Column(scale=1):
            file_input = gr.File(label="PDF 파일 선택")
            upload_button = gr.Button("📤 벡터 DB에 저장")
            show_files_button = gr.Button("📚 저장된 문서 보기")

        with gr.Column(scale=2):
            status_output = gr.Textbox(label="📢 상태 메시지")
            question_input = gr.Textbox(label="❓ 질문 입력", placeholder="궁금한 내용을 적어주세요.")
            submit_button = gr.Button("🤖 답변 받기")
            answer_output = gr.Textbox(label="📝 AI 답변")

    upload_button.click(add_pdf_to_db, inputs=file_input, outputs=status_output)
    submit_button.click(answer_question, inputs=question_input, outputs=answer_output)
    show_files_button.click(show_stored_documents, outputs=status_output)

# 벡터 DB 로드 후 실행
load_faiss_db()
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://941835b04e8cf35e5f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [1]:
#!pip uninstall numpy -y
#!pip install --no-cache-dir numpy==1.26.4
# colab에 numpy 2.버전이 설치되어 있어서 버전 충돌남

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 85.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [1]:
import numpy
print(numpy.__version__)

1.26.4


In [ ]:
import gradio
print(gradio.__version__)

3.50.2


In [ ]:
# Colab에 임베딩 모델 다운로드
#from huggingface_hub import snapshot_download
#snapshot_download(repo_id="BAAI/bge-m3", local_dir="/content/drive/MyDrive/EmbeddingModel/bge-m3")